In [90]:
import pandas as pd
import numpy as np
import matplotlib as plt
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
import heapq

In [4]:
df = pd.read_csv('GoodReads_100k_books.csv')
df.head()

,author,bookformat,desc,genre,img,isbn,isbn13,link,pages,rating,reviews,title,totalratings
0,Laurence M. Hauptman,Hardcover,Reveals that several hundred thousand Indians ...,"History,Military History,Civil War,American Hi...",https://i.gr-assets.com/images/S/compressed.ph...,002914180X,9.78E+12,https://goodreads.com/book/show/1001053.Betwee...,0,3.52,5,Between Two Fires: American Indians in the Civ...,33
1,"Charlotte Fiell,Emmanuelle Dirix",Paperback,Fashion Sourcebook - 1920s is the first book i...,"Couture,Fashion,Historical,Art,Nonfiction",https://i.gr-assets.com/images/S/compressed.ph...,1906863482,9.78E+12,https://goodreads.com/book/show/10010552-fashi...,576,4.51,6,Fashion Sourcebook 1920s,41
2,Andy Anderson,Paperback,The seminal history and analysis of the Hungar...,"Politics,History",https://i.gr-assets.com/images/S/compressed.ph...,948984147,9.78E+12,https://goodreads.com/book/show/1001077.Hungar...,124,4.15,2,Hungary 56,26
3,Carlotta R. Anderson,Hardcover,"""All-American Anarchist"" chronicles the life a...","Labor,History",https://i.gr-assets.com/images/S/compressed.ph...,814327079,9.78E+12,https://goodreads.com/book/show/1001079.All_Am...,324,3.83,1,All-American Anarchist: Joseph A. Labadie and ...,6
4,Jean Leveille,NaN,"Aujourdâ€™hui, lâ€™oiseau nous invite Ã sa ta...",NaN,https://i.gr-assets.com/images/S/compressed.ph...,2761920813,NaN,https://goodreads.com/book/show/10010880-les-o...,177,4.00,1,Les oiseaux gourmands,1


In [5]:
import spacy
spacy.cli.download('en_core_web_sm')

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [11]:
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\brend\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\brend\AppData\Roaming\nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\brend\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\brend\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\brend\AppData\Roaming\nltk_data...


True

In [6]:
nlp = spacy.load('en_core_web_sm')

In [75]:
# Get root of each of the words
REPLACE_NON_ASCII = re.compile("[^\x00-x7f]+")
REPLACE_NO_SPACE = re.compile("[;:!\'\’?,\"()\[\]]")

def preprocess(texts):
    texts = [REPLACE_NON_ASCII.sub("", row) for row in texts]
    texts = [REPLACE_NO_SPACE.sub("", row.lower()) for row in texts]
    documents = []
    for text in texts:
        documents.append(nlp(text))
    doclemmas = []
    for doc in documents:
        doclemmas.append([token.lemma_ for token in doc])
    corpus = []
    for lemmas in doclemmas:
        corpus.append(' '.join(lemmas).lower())
    return corpus

def get_lemmed(corpus):
    lemmatizer = WordNetLemmatizer()
    corpus = [' '.join([lemmatizer.lemmatize(word, pos='v') for word in review.split()]) for review in corpus]
    return [' '.join([lemmatizer.lemmatize(word, pos='n') for word in review.split()]) for review in corpus]

def preprocess(texts):
    # print(texts)
    # for text in texts:
    #     if len(str(text)) == 0:
    #         continue    
    for i in range(len(texts)):
        texts[i] = str(texts[i])
        if texts[i] == "nan":
            texts[i] = ""
    REPLACE_NON_ASCII = re.compile("[^\x00-\x7F]+")
    REPLACE_NO_SPACE = re.compile("[;:!\'\’?,\"()\[\]]")
    texts = [REPLACE_NON_ASCII.sub("", row) for row in texts]
    texts = [REPLACE_NO_SPACE.sub("", row.lower()) for row in texts]
    texts = get_lemmed(texts)
    texts = [x.lower() for x in texts]
    return texts


In [14]:
sample = "This is a sample text for testing"
clean_text = preprocess([sample])
print(clean_text)

['this be a sample text for test']


In [12]:
df['desc']

0        Reveals that several hundred thousand Indians ...
1        Fashion Sourcebook - 1920s is the first book i...
2        The seminal history and analysis of the Hungar...
3        "All-American Anarchist" chronicles the life a...
4        Aujourdâ€™hui, lâ€™oiseau nous invite Ã  sa ta...
                               ...                        
99995    Design custom printed circuit boards with EAGL...
99996    Redefining tradition in learning accounting. ,...
99997    In these warm reflections on his own growth as...
99998    John Brown is a man of many legacies, from her...
99999    Did you know that interest in UFO research inc...
Name: desc, Length: 100000, dtype: object

In [95]:
df2 = df.loc[:10000]
orig = df2['desc'].tolist()
text_clean = preprocess(orig)

df2['desc-clean'] = text_clean
df2['desc-clean']

# df2 = df[:1]
# df2['desc-clean'] = df2['desc']
# df2['desc-clean'] =df2['desc-clean'].apply(lambda x: preprocess(str(x)))
# df2['desc-clean']
# corpus = preprocess(df['desc'])

C:\Users\brend\AppData\Local\Temp\ipykernel_16632\2282408193.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['desc-clean'] = text_clean


0        reveal that several hundred thousand indian be...
1        fashion sourcebook - 1920s be the first book i...
2        the seminal history and analysis of the hungar...
3        all-american anarchist chronicle the life and ...
4        aujourdhui loiseau nous invite sa table table ...
                               ...                        
9996     the essay in this collection address how the v...
9997     from the author of the acclaim best seller the...
9998     abraham lincoln stand out in a crowd a much fo...
9999     present the detail and dramatic ritual for eac...
10000    winner of the lincoln prizelincoln at cooper u...
Name: desc-clean, Length: 10001, dtype: object

In [27]:
print(df['desc'].iloc[30])

nan


In [96]:
cv = CountVectorizer(stop_words='english', ngram_range=(1,2))
stopwords = []
for word in cv.get_stop_words():
    if word != 'not':
        stopwords.append(word)
cv = CountVectorizer(stop_words=stopwords, ngram_range=(1,2))
X = cv.fit_transform(df2['desc-clean'])
X

<10001x641624 sparse matrix of type '<class 'numpy.int64'>'
	with 1258710 stored elements in Compressed Sparse Row format>

In [97]:
cosine_sim = linear_kernel(X, X)
print(cosine_sim)

[[ 35.   0.   1. ...   6.   0.   0.]
 [  0. 205.   1. ...   6.   2.   5.]
 [  1.   1.  27. ...   1.   1.   0.]
 ...
 [  6.   6.   1. ... 205.   2.  40.]
 [  0.   2.   1. ...   2. 129.   0.]
 [  0.   5.   0. ...  40.   0. 307.]]


In [106]:
myStrings = ["artificial intelligence ethics"]
myStrings = preprocess(myStrings)
print(myStrings)

new_x = cv.transform(myStrings)
print(new_x)

cosine_sim2 = linear_kernel(new_x, X)
print(cosine_sim2)

most_similar_index = np.argmax(cosine_sim2)
most_similar_book = orig[most_similar_index]
print(most_similar_book)

['artificial intelligence ethic']
  (0, 37628)	1
  (0, 37632)	1
  (0, 191546)	1
  (0, 289993)	1
[[0. 0. 0. ... 0. 0. 0.]]
John Sowa integrates logic, philosophy, linguistics, and computer science into this study of knowledge and its various models and implementations. His definitive new book shows how techniques of artificial intelligence, database design, and object-oriented programming help make knowledge explicit in a form that computer systems can use. The first three chapters are devoted to logic, ontology, and computable models of reality. Remaining chapters apply theories to the analysis of problems stated in ordinary language, and their translation to computable form. The text is self-contained, with each new idea defined when first mentioned; all formalism is developed in the body of the text or summarized in an appendix. Knowledge Representation is appropriate for advanced undergraduate and graduate students in computer science, as well as philosophy and linguistics students 

In [107]:
top_n = 10

top_similar_indices = heapq.nlargest(top_n, range(len(cosine_sim2[0])), cosine_sim2[0].take)
print("Top ", top_n, " most similar strings:")
for idx in top_similar_indices:
    print("Score: ", cosine_sim2[0, idx], ">> ", df['title'].iloc[idx], " >>> ", orig[idx])
    print()


Top  10  most similar strings:
Score:  6.0 >>  Knowledge Representation: Logical, Philosophical, and Computational Foundations  >>>  John Sowa integrates logic, philosophy, linguistics, and computer science into this study of knowledge and its various models and implementations. His definitive new book shows how techniques of artificial intelligence, database design, and object-oriented programming help make knowledge explicit in a form that computer systems can use. The first three chapters are devoted to logic, ontology, and computable models of reality. Remaining chapters apply theories to the analysis of problems stated in ordinary language, and their translation to computable form. The text is self-contained, with each new idea defined when first mentioned; all formalism is developed in the body of the text or summarized in an appendix. Knowledge Representation is appropriate for advanced undergraduate and graduate students in computer science, as well as philosophy and linguistic